### Passo 4 - Implementação do Modelo

In [20]:
# Importação das Bibliotecas
import os
from io import BytesIO
import pandas as pd
import joblib
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

# Definição das Variáveis
load_dotenv("../.env")
connection_string = os.getenv("CONNECTION_STRING")
model_container_name = "models"
model_blob_name = "xgboost_best.pkl"
novo_pokemon_csv = "../data/novo_pokemon.csv"
output_csv = "../data/novo_pokemon_predito.csv"

# Conexão com Azure Blob
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_client = blob_service_client.get_blob_client(container=model_container_name, blob=model_blob_name)

# Carregando o modelo salvo (dicionário com modelo e LabelEncoder)
model_bytes = blob_client.download_blob().readall()
loaded_obj = joblib.load(BytesIO(model_bytes))

model = loaded_obj["model"]
label_encoder = loaded_obj.get("label_encoder")  # opcional, caso tenha sido salvo

print("# Modelo carregado com sucesso!")

# Carregar CSV de novos Pokémon
if not os.path.exists(novo_pokemon_csv):
    raise FileNotFoundError(f"O arquivo '{novo_pokemon_csv}' não foi encontrado.")

df_new = pd.read_csv(novo_pokemon_csv)
required_features = ["base_total", "capture_rate", "is_legendary"]

# Verificar colunas necessárias
missing = [col for col in required_features if col not in df_new.columns]
if missing:
    raise ValueError(f"O CSV não contém as colunas necessárias: {missing}")

X_new = df_new[required_features]
print("# Colunas corretas detectadas para previsão:")
display(X_new.head())

# Realizando a previsão
predictions_encoded = model.predict(X_new)

# Converter para os valores originais de experience_growth
growth_map_reverse = {
    0: 600000,    # Fast
    1: 800000,    # Medium Fast
    2: 1000000,   # Medium Slow
    3: 1059860,   # Medium Slow (outra variante)
    4: 1640000,   # Slow
    5: 6000000    # Fluctuating
}

# Se LabelEncoder foi usado no treinamento
if label_encoder:
    predictions_numeric = label_encoder.inverse_transform(predictions_encoded)
else:
    predictions_numeric = [growth_map_reverse.get(p, p) for p in predictions_encoded]

df_new["Predicted_Experience_Growth"] = predictions_numeric

print("# Previsões realizadas com sucesso!")
display(df_new.head())

# Salvar resultado localmente
df_new.to_csv(output_csv, index=False)
print(f"# Resultado salvo em '{output_csv}'")


# Modelo carregado com sucesso!
# Colunas corretas detectadas para previsão:


,base_total,capture_rate,is_legendary
0,318,45,0
1,525,45,0
2,309,45,0
3,534,45,0
4,308,45,0


# Previsões realizadas com sucesso!


,name,base_total,capture_rate,is_legendary,Predicted_Experience_Growth
0,Orchynx,318,45,0,Medium Fast
1,Metalynx,525,45,0,Medium Fast
2,Raptorch,309,45,0,Medium Fast
3,Archilles,534,45,0,Medium Fast
4,Eletux,308,45,0,Medium Fast


# Resultado salvo em '../data/novo_pokemon_predito.csv'
